In [1]:
import pandas as pd
import requests
import re
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
import json

current_dir = ""

In [2]:
df = pd.read_csv(current_dir + "paperInfo.csv")
df

,Key,Year,Url,Publication Title,Title,Abstract（英文）,Keywords,标题,Abstract（中文）,关键词,可用数据集,源代码,备注（如改进方向等）
0,C8VSYBES,2023,https://doi.org/10.1109/ASE56229.2023.00213,ASE 2023,CPA-DF: A Tool for Configurable Interval Analy...,"Software verification is challenging, and auxi...","Program Analysis,Data Flow Analysis,Software M...",CPA-DF：一种可配置区间分析工具，用于增强程序验证,软件验证具有挑战性，使用辅助程序不变量来提高验证方法的有效性。例如，CPACHECKER中的...,程序分析，数据流分析，软件模型检查，区间分析，动态精度调整，不变量生成,NaN,NaN,NaN
1,8PBLFHI9,2023,https://doi.org/10.1109/ASE56229.2023.00146,ASE 2023,Provengo: A Tool Suite for Scenario Driven Mod...,"We present Provengo, a comprehensive suite of ...","Testing,Software engineering,Business",Provengo：一个用于场景驱动的基于模型的测试的工具套件,我们介绍了Provengo，这是一套全面的工具，旨在促进场景驱动的基于模型的测试（SDMBT...,测试，软件工程，商业,NaN,NaN,NaN
2,YXIZFW3F,2023,https://doi.org/10.1109/ASE56229.2023.00065,ASE 2023,"Better Patching Using LLM Prompting, via Self-...",Large Language models (LLMs) can be induced to...,"LLMs,Self-consistency,Program Repair",通过自一致性使用LLM提示进行更好的修补,大型语言模型（LLM）可以通过“少量”提示（包括说明性问题解决方案示例）来解决非琐碎的问题。...,LLM，自一致性，程序修复,NaN,NaN,NaN
3,G9XMACX6,2023,https://doi.org/10.1109/ASE56229.2023.00216,ASE 2023,Optimizing Continuous Development by Detecting...,Continuous development (CD) helps developers q...,"Runtime,Static analysis,Writing,Chatbots,Softw...",通过检测和防止不必要的内容生成优化持续开发,持续开发（CD）帮助开发人员快速发布和更新软件。为了制定CD，开发人员自定义他们的CD构建以...,聊天机器人，静态分析，写作，软件，任务分析，测试,NaN,NaN,NaN
4,EUT579M7,2023,https://doi.org/10.1109/ASE56229.2023.00092,ASE 2023,Modeling Programmer Attention as Scanpath Pred...,This paper launches a new effort at modeling p...,"scanpath prediction,human attention,eye tracki...",将程序员注意力建模为扫描路径预测,本文通过预测眼动扫描路径，对程序员的注意力进行了建模。程序员注意力是指人们在执行编程任务时所...,扫描路径预测，人类注意力，眼睛跟踪，神经网络，人工智能,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,HAB6J8JW,2023,https://doi.org/10.1145/3563212,TOSEM 2023,Precise Quantitative Analysis of Binarized Neu...,"As a new programming paradigm, neural-network-...","Computing methodologies,Machine learning,Machi...",二值化神经网络的精确定量分析：一种基于BDD的方法,作为一种新的编程范式，基于神经网络的机器学习已经将其应用扩展到许多现实问题中。由于神经网络的...,计算方法，机器学习，机器学习方法，神经网络，安全和隐私，软件和应用程序安全，软件安全工程，软...,NaN,NaN,NaN
1156,79Z7UIKV,2023,https://doi.org/10.1145/3571847,TOSEM 2023,Securing the Ethereum from Smart Ponzi Schemes...,Malware detection approaches have been extensi...,"Security and privacy,Intrusion/anomaly detecti...",从智能庞氏骗局中保护以太坊：使用静态特征的识别,针对传统软件系统，恶意软件检测方法已被广泛研究。然而，区块链技术的发展推动了一种新型软件系统...,安全和隐私，入侵/异常检测和恶意软件缓解，恶意软件及其缓解，软件和应用程序安全,NaN,NaN,NaN
1157,BKJKTU8E,2023,https://doi.org/10.1145/3582569,TOSEM 2023,Client-Specific Upgrade Compatibility Checking...,"Modern software systems are complex, and they ...","Software and its engineering,Software creation...",通过知识引导的发现进行特定于客户端的升级兼容性检查,现代软件系统是复杂的，它们严重依赖于不同团队和组织开发的外部库。由于库升级引起的不兼容性问题...,软件及其工程，软件创建和管理，软件开发后问题，维护软件，软件演化，软件符号和工具，软件库和存储库,NaN,NaN,NaN
1158,CAJPIICK,2023,https://doi.org/10.1145/3544792,TOSEM 2023,Efficient and Effective Feature Space Explorat...,Assessing the quality of Deep Learning (DL) sy...,"Software and its engineering,Software creation...",用于测试深度学习系统的高效特征空间探索,评估深度学习（DL）系统的质量至关重要，因为它们越来越多地被用于安全关键领域。研究人员提出了...,软件及其工程，软件创建和管理，软件验证和确认，软件缺陷分析，软件测试和调试,NaN,NaN,NaN


In [3]:
keyword_dic = {}
def statisticsCount(row):
  if isinstance(row['Keywords'], str):
    keywords = row['Keywords'].split(',')
    for keyword in keywords:
      if keyword in keyword_dic.keys():
        keyword_dic[keyword] = keyword_dic[keyword] + 1
      else:
        keyword_dic[keyword] = 1
  return row
df.apply(statisticsCount, axis=1)

,Key,Year,Url,Publication Title,Title,Abstract（英文）,Keywords,标题,Abstract（中文）,关键词,可用数据集,源代码,备注（如改进方向等）
0,C8VSYBES,2023,https://doi.org/10.1109/ASE56229.2023.00213,ASE 2023,CPA-DF: A Tool for Configurable Interval Analy...,"Software verification is challenging, and auxi...","Program Analysis,Data Flow Analysis,Software M...",CPA-DF：一种可配置区间分析工具，用于增强程序验证,软件验证具有挑战性，使用辅助程序不变量来提高验证方法的有效性。例如，CPACHECKER中的...,程序分析，数据流分析，软件模型检查，区间分析，动态精度调整，不变量生成,NaN,NaN,NaN
1,8PBLFHI9,2023,https://doi.org/10.1109/ASE56229.2023.00146,ASE 2023,Provengo: A Tool Suite for Scenario Driven Mod...,"We present Provengo, a comprehensive suite of ...","Testing,Software engineering,Business",Provengo：一个用于场景驱动的基于模型的测试的工具套件,我们介绍了Provengo，这是一套全面的工具，旨在促进场景驱动的基于模型的测试（SDMBT...,测试，软件工程，商业,NaN,NaN,NaN
2,YXIZFW3F,2023,https://doi.org/10.1109/ASE56229.2023.00065,ASE 2023,"Better Patching Using LLM Prompting, via Self-...",Large Language models (LLMs) can be induced to...,"LLMs,Self-consistency,Program Repair",通过自一致性使用LLM提示进行更好的修补,大型语言模型（LLM）可以通过“少量”提示（包括说明性问题解决方案示例）来解决非琐碎的问题。...,LLM，自一致性，程序修复,NaN,NaN,NaN
3,G9XMACX6,2023,https://doi.org/10.1109/ASE56229.2023.00216,ASE 2023,Optimizing Continuous Development by Detecting...,Continuous development (CD) helps developers q...,"Runtime,Static analysis,Writing,Chatbots,Softw...",通过检测和防止不必要的内容生成优化持续开发,持续开发（CD）帮助开发人员快速发布和更新软件。为了制定CD，开发人员自定义他们的CD构建以...,聊天机器人，静态分析，写作，软件，任务分析，测试,NaN,NaN,NaN
4,EUT579M7,2023,https://doi.org/10.1109/ASE56229.2023.00092,ASE 2023,Modeling Programmer Attention as Scanpath Pred...,This paper launches a new effort at modeling p...,"scanpath prediction,human attention,eye tracki...",将程序员注意力建模为扫描路径预测,本文通过预测眼动扫描路径，对程序员的注意力进行了建模。程序员注意力是指人们在执行编程任务时所...,扫描路径预测，人类注意力，眼睛跟踪，神经网络，人工智能,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,HAB6J8JW,2023,https://doi.org/10.1145/3563212,TOSEM 2023,Precise Quantitative Analysis of Binarized Neu...,"As a new programming paradigm, neural-network-...","Computing methodologies,Machine learning,Machi...",二值化神经网络的精确定量分析：一种基于BDD的方法,作为一种新的编程范式，基于神经网络的机器学习已经将其应用扩展到许多现实问题中。由于神经网络的...,计算方法，机器学习，机器学习方法，神经网络，安全和隐私，软件和应用程序安全，软件安全工程，软...,NaN,NaN,NaN
1156,79Z7UIKV,2023,https://doi.org/10.1145/3571847,TOSEM 2023,Securing the Ethereum from Smart Ponzi Schemes...,Malware detection approaches have been extensi...,"Security and privacy,Intrusion/anomaly detecti...",从智能庞氏骗局中保护以太坊：使用静态特征的识别,针对传统软件系统，恶意软件检测方法已被广泛研究。然而，区块链技术的发展推动了一种新型软件系统...,安全和隐私，入侵/异常检测和恶意软件缓解，恶意软件及其缓解，软件和应用程序安全,NaN,NaN,NaN
1157,BKJKTU8E,2023,https://doi.org/10.1145/3582569,TOSEM 2023,Client-Specific Upgrade Compatibility Checking...,"Modern software systems are complex, and they ...","Software and its engineering,Software creation...",通过知识引导的发现进行特定于客户端的升级兼容性检查,现代软件系统是复杂的，它们严重依赖于不同团队和组织开发的外部库。由于库升级引起的不兼容性问题...,软件及其工程，软件创建和管理，软件开发后问题，维护软件，软件演化，软件符号和工具，软件库和存储库,NaN,NaN,NaN
1158,CAJPIICK,2023,https://doi.org/10.1145/3544792,TOSEM 2023,Efficient and Effective Feature Space Explorat...,Assessing the quality of Deep Learning (DL) sy...,"Software and its engineering,Software creation...",用于测试深度学习系统的高效特征空间探索,评估深度学习（DL）系统的质量至关重要，因为它们越来越多地被用于安全关键领域。研究人员提出了...,软件及其工程，软件创建和管理，软件验证和确认，软件缺陷分析，软件测试和调试,NaN,NaN,NaN


In [4]:
sorted_dict = sorted(keyword_dic.items(), key=lambda x: x[1], reverse=True)
sorted_dict

[('Software and its engineering', 443),
 ('Software creation and management', 385),
 ('Software verification and validation', 257),
 ('Software defect analysis', 188),
 ('Software notations and tools', 180),
 ('Software testing and debugging', 174),
 ('Computing methodologies', 115),
 ('Software organization and properties', 113),
 ('Machine learning', 89),
 ('Security and privacy', 86),
 ('Social and professional topics', 80),
 ('Software post-development issues', 76),
 ('Theory of computation', 66),
 ('Professional topics', 62),
 ('Software and application security', 60),
 ('Software configuration management and version control systems', 57),
 ('General and reference', 56),
 ('Human-centered computing', 55),
 ('Software functional properties', 54),
 ('Management of computing and information systems', 54),
 ('Information systems', 49),
 ('Cross-computing tools and techniques', 44),
 ('Semantics and reasoning', 42),
 ('Software maintenance tools', 41),
 ('Software development technique

In [8]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('max_colwidth',100)

df = df.dropna(subset=['Abstract（英文）'])
commit_df = df[df['Title'].str.lower().str.contains("commit message")]
print(len(commit_df))
commit_df

5


,Key,Year,Url,Publication Title,Title,Abstract（英文）,Keywords,标题,Abstract（中文）,关键词,可用数据集,源代码,备注（如改进方向等）
95,7FM8PQAR,2023,https://doi.org/10.1109/ASE56229.2023.00078,ASE 2023,From Commit Message Generation to History-Awar...,Commit messages are crucial to software develo...,"Computer languages,Filtering,Focusing,Writing,...",从提交消息生成到历史感知提交消息完成,提交消息对软件开发至关重要，使开发人员能够跟踪更改并进行有效协作。尽管它们很有用，但大多数提...,计算机语言，过滤，聚焦，写作，软件，历史，可靠性,NaN,NaN,NaN
156,EA26F5HY,2023,https://doi.org/10.1109/ASE56229.2023.00050,ASE 2023,Delving into Commit-Issue Correlation to Enhan...,Commit message generation (CMG) is a challengi...,"Commit Message Generation,Dataset Construction...",深入研究提交问题相关性以增强提交消息生成模型,提交消息生成（CMG）是自动化软件工程中一项具有挑战性的任务，旨在为提交生成代码更改的自然语...,提交消息生成，数据集构建，代码表示学习,NaN,NaN,NaN
238,3L6D7IAV,2023,https://doi.org/10.1109/ICSE48619.2023.00075,ICSE 2023,Revisiting Learning-based Commit Message Gener...,Commit messages summarize code changes and hel...,"Commit Message Generation,Deep Learning,Patter...",重新审视基于学习的提交消息生成,提交消息总结代码更改并帮助开发人员理解其意图。为了减轻人类在编写提交消息方面的努力，研究人员...,提交消息生成，深度学习，基于模式,NaN,NaN,NaN
290,6WZZD6M7,2023,https://doi.org/10.1109/ICSE48619.2023.00076,ICSE 2023,Commit Message Matters: Investigating Impact a...,Commit messages play an important role in comm...,"Commit message quality,software defect pronene...",提交消息事项：调查提交消息质量的影响和演变,提交消息在开发人员之间的沟通中起着重要作用。为了衡量提交消息的质量，研究人员定义了什么在语义...,承诺消息质量，软件缺陷倾向，实证分析,NaN,NaN,NaN
642,YQSK3RMA,2023,https://doi.org/10.1145/3597926.3598096,ISSTA 2023,COME: Commit Message Generation with Modificat...,Commit messages concisely describe code change...,"Software and its engineering,Software notation...",COME：嵌入修改的提交消息生成,提交消息简洁地描述了自然语言中的代码更改，对程序的理解和维护很重要。先前的研究提出了一些自动...,软件及其工程，软件符号和工具，软件维护工具,NaN,NaN,NaN
